# Computer Vision (Image operators and filters)

By the end of this lab, you will get hands on experience working with:

*   Image Handling
*   Image Manipulation
*   Histogram and Histogram Equalization
*   Basic filtering techniques

<!-- ### **Remember this is a graded exercise.** -->

**Reminder**:

*   For every plot, make sure you provide appropriate titles, axis labels, legends, wherever applicable.
*   Add sufficient comments and explanations wherever necessary.

---


In [ ]:
# Loading necessary libraries (Feel free to add new libraries if you need for any computation)
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from skimage import data, exposure, filters, io, morphology, color

# Channels and color spaces

### **Exercise: Image Creation and Color Manipulation**

*   Create a 100 x 100 image for each of the below visualization


*   Visualize the created images in a 1 x 3 subplot using matplotlib.


In [ ]:
# solution
image_1 = np.zeros((100, 100))
image_1[:, 50:] = 1

image_2 = np.zeros((100, 100))
image_2[50:, :] = 1

image_3 = np.zeros((100, 100))
image_3[0:50, 0:50] = 1


figs, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(image_1, cmap='gray')
axes[0].set_title("Image 1- Left Black Right White")
axes[0].axis('on')

axes[1].imshow(image_2, cmap='gray')
axes[1].set_title("Image 2- Top Black Bottom White")
axes[1].axis('on')

axes[2].imshow(image_3, cmap='gray')
axes[2].set_title("Image 3 - Top Left Corner White and Rest Black")
axes[2].axis('on')



plt.tight_layout()
plt.show()

*   Use the above three images to create the following image

*Hint: Remember channels and color spaces*

In [ ]:
# solution
image = np.zeros((100, 100, 3))

#Top-left: Blue
image[:50, :50, 2] = 1 

#Top-right: Red
image[:50, 50:, 0] = 1

#Bottom-left: Green
image[50:, :50, 1] = 1  

#Bottom-right: Yellow
image[50:, 50:, 0] = 1  
image[50:, 50:, 1] = 1 

plt.imshow(image)
plt.axis('off')
plt.title("Four Corners: Red, Blue, Green, Yellow")
plt.show()

### **Exercise: Color Manipulation**

*   Read the image 'sillas.jpg' from the images folder

*   Extract individual channels and plot them using matplotlib subplot.



In [ ]:
# solution
sillas_image = io.imread('images/sillas.jpg')

#Extract channels
red_channel = sillas_image[:, :, 0]
green_channel = sillas_image[:, :, 1]
blue_channel = sillas_image[:, :, 2]

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

#Display the original image
axes[0].imshow(sillas_image)
axes[0].set_title("Original Image")
axes[0].axis('off')

#Display the Red channel
axes[1].imshow(red_channel, cmap='Reds')
axes[1].set_title("Red Channel")
axes[1].axis('off')

#Display the Green channel
axes[2].imshow(green_channel, cmap='Greens')
axes[2].set_title("Green Channel")
axes[2].axis('off')

#Display the Blue channel
axes[3].imshow(blue_channel, cmap='Blues')
axes[3].set_title("Blue Channel")
axes[3].axis('off')

plt.tight_layout()
plt.show()

*   The color **red** looks too bright for the eyes. Isn't it?? Lets change the color and see how it appears.
    *    Create a new image where everything that is **'red' is changed to 'blue'**.
*   Visualize the original image and the created image using matplotlib subplot.

In [ ]:
# solution
#Create a copy of the image to modify
copy_image = np.copy(sillas_image)
copy_image[:, :, 0] = blue_channel

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

#Original image
axes[0].imshow(sillas_image)
axes[0].set_title("Original Image")
axes[0].axis('off')

#Modified image
axes[1].imshow(copy_image)
axes[1].set_title("Modified Image (Red to Blue)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Image Manipulation

### **Exercise: Image Operators**

*   You can find images 'model.png' and 'coat.png' in the images folder (First two images of the below visualization). Your task is to create an image from the given two images such a way that the model is wearing the coat (Third image in the visualization).
*   You can also find different textures in the images folder. Your task is to change the coat texture to any one of the given textures.
*   Visualize the images similar to the given visualization.

*Hint: Think masks!!!*


In [ ]:
model_image = io.imread('images/model.png')
coat_image = io.imread('images/coat.png')
texture_image = io.imread('images/texture.png')

#Remove alpha channels if they exist
if model_image.shape[2] == 4:
    model_image = model_image[:, :, :3]  # Remove alpha channel if present
if coat_image.shape[2] == 4:
    coat_image = coat_image[:, :, :3]  # Remove alpha channel if present
if texture_image.shape[2] == 4:
    texture_image = texture_image[:, :, :3]  # Remove alpha channel if present

#Resize coat and texture to match the model image
coat_resize = cv.resize(coat_image, (model_image.shape[1], model_image.shape[0]))
texture_resize = cv.resize(texture_image, (model_image.shape[1], model_image.shape[0]))

#Create mask for the coat using grayscale
gray_coat = cv.cvtColor(coat_resize, cv.COLOR_RGB2GRAY)
_, mask = cv.threshold(gray_coat, 1, 255, cv.THRESH_BINARY)

#Expand the mask to 3 channels
mask_3d = np.stack([mask] * 3, axis=-1)

#Apply the coat on the model image
model_with_coat = model_image.copy()
model_with_coat[mask_3d == 255] = coat_resize[mask_3d == 255]

#Apply the texture to the coat region (masked area)
coat_with_texture = coat_resize.copy()
coat_with_texture[mask == 255] = texture_resize[mask == 255]

#Overlay the textured coat onto the model
model_with_textured_coat = model_image.copy()
model_with_textured_coat[mask_3d == 255] = coat_with_texture[mask_3d == 255]


figs, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(coat_image)
axes[0].set_title("Coat")
axes[0].axis('on')

axes[1].imshow(model_image)
axes[1].set_title("Model")
axes[1].axis('on')

axes[2].imshow(model_with_yellow_coat)
axes[2].set_title("Model with Coat")
axes[2].axis('on')

axes[3].imshow(model_with_textured_coat)
axes[3].set_title("Model with Textured Coat")
axes[3].axis('on')

plt.tight_layout()
plt.show()

# Contrast Enhancement

### **Exercise: Histogram Computation**

*   Read the **'astronaut' image** from data module.
*   Convert the image to grayscale.
*   Compute the **histogram of the image.** *Hint: histogram function is available in skimage.exposure package*
*   Plot the histogram using matplotlib plot.




In [ ]:
# solution
astronaut_image = data.astronaut()

#Grayscale
grayscale_astronaut = color.rgb2gray(astronaut_image)

hist, bin_centers = exposure.histogram(grayscale_astronaut)

#Plot histogram
plt.figure(figsize=(10, 6))
plt.plot(bin_centers, hist, lw=2)
plt.title('Histogram of Grayscale Astronaut Image')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

*   Change the bin count to 8 and compute the histogram of the image and plot the computed histogram using matplotlib plot.

In [ ]:
# solution
#Compute the histogram with 8 bins
hist, bin_centers = exposure.histogram(grayscale_astronaut, nbins=8)

#Plot histogram
plt.figure(figsize=(10, 6))
plt.bar(bin_centers, hist, width=0.1, color='blue', edgecolor='black')
plt.title('Histogram of Grayscale Astronaut Image (8 bins)')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()



*   What happens when you change the bin count? Does your inference change based on the bin count? If yes, then how do you define the correct bin count.
*   What happens when the bin count is very low and what happens when it is very high?



**Solution**

Changing the bin count in a histogram affects how the data is grouped and visualized. A low bin count (e.g., 8 bins) makes the histogram coarser, losing detailed variations in pixel intensities, but provides a rough overview of the distribution. This can be useful for general trends, but finer details may be missed. A high bin count (e.g., 256 or more) captures more detailed variations, revealing subtle differences in pixel intensity, but can lead to overfitting by emphasizing noise and making the histogram harder to interpret. In other words, too few bins may oversimplify the data, while too many can complicate interpretation.


*   Compute histogram of the color image (without converting it to grayscale).
*   Plot the total histogram and also histogram for each channel (show it in a single plot with differnt legends for each histogram).


In [ ]:
#solution
astronaut_image = data.astronaut()

#Split the image into Red, Green, and Blue channels
red_channel = astronaut_image[:, :, 0]  # Red channel
green_channel = astronaut_image[:, :, 1]  # Green channel
blue_channel = astronaut_image[:, :, 2]  # Blue channel

#Compute histograms for each channel
hist_red, bin_centers_red = exposure.histogram(red_channel)
hist_green, bin_centers_green = exposure.histogram(green_channel)
hist_blue, bin_centers_blue = exposure.histogram(blue_channel)

plt.figure(figsize=(10, 6))

plt.plot(bin_centers_red, hist_red, color='red', label='Red Channel', lw=2)
plt.plot(bin_centers_green, hist_green, color='green', label='Green Channel', lw=2)
plt.plot(bin_centers_blue, hist_blue, color='blue', label='Blue Channel', lw=2)

# Plot histogram
total_hist = hist_red + hist_green + hist_blue
plt.plot(bin_centers_red, total_hist, color='black', label='Total Histogram', lw=2, linestyle='--')
plt.title('Histogram of Color Image and Each Channel')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

### **Exercise: Histogram Equalization**

*   Read 'aquatermi_lowcontrast.jpg' image from the images folder.
*   Compute the histogram of the image.
*   Perform histogram equalization of the image to enhance the contrast. *Hint: Use equalize_hist function available in skimage.exposure*
*   Also compute histogram of the equalized image.
*   Use 2 x 2 subplot to show the original image and the enhanced image along with the corresponding histograms.



In [ ]:
# solution
aquatermi_image = io.imread('images/aquatermi_lowcontrast.jpg')

#Compute the histogram of the original image
hist_original, bin_centers_original = exposure.histogram(aquatermi_image)

#Perform histogram equalization
equalized_image = exposure.equalize_hist(aquatermi_image)

#Compute the histogram of the equalized image
hist_equalized, bin_centers_equalized = exposure.histogram(equalized_image)

#Create 2x2 subplot for displaying the images and histograms
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

#Display original image
axes[0, 0].imshow(aquatermi_image, cmap='gray')
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

#Display histogram of the original image
axes[0, 1].plot(bin_centers_original, hist_original, color='black')
axes[0, 1].set_title('Histogram of Original Image')
axes[0, 1].set_xlabel('Pixel Intensity')
axes[0, 1].set_ylabel('Frequency')

#Display equalized image
axes[1, 0].imshow(equalized_image, cmap='gray')
axes[1, 0].set_title('Equalized Image')
axes[1, 0].axis('off')

#Display histogram of the equalized image
axes[1, 1].plot(bin_centers_equalized, hist_equalized, color='black')
axes[1, 1].set_title('Histogram of Equalized Image')
axes[1, 1].set_xlabel('Pixel Intensity')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


*   The above function in skimage.exposure uses cdf and interpolation technique to normalize the histogram. How is it different from linear contrast stretch?


**Solution**

Histogram equalization and linear contrast stretching both enhance image contrast but differ in their approaches. Histogram equalization redistributes pixel intensities to achieve a more uniform histogram, enhancing contrast non-linearly across the image. This can lead to over-enhancement in well-contrasted areas. On the other hand, linear contrast stretching maps pixel intensities between the minimum and maximum values, providing a simple, linear contrast boost. While histogram equalization adapts to the image's distribution, linear contrast stretching offers a more predictable and controlled enhancement.

### **Exercise: Linear Contrast Stretch**

*   Write a function to compute the linear contrast stretch (Do not use an inbuilt function). 
*   Provide grayscale image array and bin count as parameters to the function and return the enhanced image array.
*   Use a 2 x 2 plot to visualize the original image, histogram, enhanced image and the corresponding histogram.



In [ ]:
# solution
#Function to perform linear contrast stretch
def linear_contrast_stretch(image, bin_count=256):
    #Find the minimum and maximum pixel values in the image
    I_min = np.min(image)
    I_max = np.max(image)
    
    #Apply linear contrast stretching formula
    enhanced_image = ((image - I_min) / (I_max - I_min) * 255).astype(np.uint8)
    
    return enhanced_image

#Read a grayscale image (use an example)
image = io.imread('images/aquatermi_lowcontrast.jpg')
if image.ndim == 3:
    image = np.mean(image, axis=2)

#Perform linear contrast stretch
enhanced_image = linear_contrast_stretch(image)

#Compute histograms
hist_original, bin_centers = exposure.histogram(image, nbins=256)
hist_enhanced, _ = exposure.histogram(enhanced_image, nbins=256)

#Create 2x2 plot
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

#Plot original image
axes[0, 0].imshow(image, cmap='gray')
axes[0, 0].set_title("Original Image")
axes[0, 0].axis('off')

#Plot histogram of original image
axes[0, 1].plot(bin_centers, hist_original, color='black')
axes[0, 1].set_title("Original Histogram")
axes[0, 1].set_xlabel("Pixel Intensity")
axes[0, 1].set_ylabel("Frequency")

#Plot enhanced image
axes[1, 0].imshow(enhanced_image, cmap='gray')
axes[1, 0].set_title("Enhanced Image")
axes[1, 0].axis('off')

#Plot histogram of enhanced image
axes[1, 1].plot(bin_centers, hist_enhanced, color='black')
axes[1, 1].set_title("Enhanced Histogram")
axes[1, 1].set_xlabel("Pixel Intensity")
axes[1, 1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

# Filters

### **Exercise: Mean Filter**

*   Load the **coins** image from the data module.
*   Define a disk structuring element (selem) of radius 20. *Hint: Structuring elements are defined in the skimage.morphology module*
*   Use mean filter using the created selem. *Hint: The mean filter is available in skimage.filters.rank module*
*   Increase the radius of the selem by 10 and apply the mean filter.
*   Reduce the radius of the selem by 10 and apply the mean filter.
*   Visualize all the smoothened images along with the original image.




In [ ]:
# solution
coins_image = data.coins()

#Define a disk structuring element with radius 20
selem_20 = morphology.disk(20)

#Apply mean filter with radius 20
smoothed_20 = filters.rank.mean(coins_image, selem_20)

#Increase radius of the structuring element by 10 (radius 30)
selem_30 = morphology.disk(30)

#Apply mean filter with radius 30
smoothed_30 = filters.rank.mean(coins_image, selem_30)

#Reduce radius of the structuring element by 10 (radius 10)
selem_10 = morphology.disk(10)

# Apply mean filter with radius 10
smoothed_10 = filters.rank.mean(coins_image, selem_10)

#Plot images
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

#Display original image
axes[0, 0].imshow(coins_image, cmap='gray')
axes[0, 0].set_title("Original Image")
axes[0, 0].axis('off')

#Display smoothed image with radius 20
axes[0, 1].imshow(smoothed_20, cmap='gray')
axes[0, 1].set_title("Smoothed Image (Radius 20)")
axes[0, 1].axis('off')

#Display smoothed image with radius 30
axes[1, 0].imshow(smoothed_30, cmap='gray')
axes[1, 0].set_title("Smoothed Image (Radius 30)")
axes[1, 0].axis('off')

#Display smoothed image with radius 10
axes[1, 1].imshow(smoothed_10, cmap='gray')
axes[1, 1].set_title("Smoothed Image (Radius 10)")
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

*   Use different selem (square, rectangle, star, diamond) to view the behaviour of the mean filter (It is not necessary to repeat with different sizes; it is sufficient to show the one with optimal parameter).
*   Create a 2 x n subplot to show the selem in the first row and the corresponding smoothened image in the second row.

In [ ]:
# solution
coins_image = data.coins()

#Define different structuring elements
selem_square = morphology.square(20)  # Square with side 20
selem_rectangle = morphology.rectangle(20, 40)  # Rectangle with dimensions 20x40
selem_star = morphology.star(20)  # Star with radius 20
selem_diamond = morphology.diamond(20)  # Diamond with radius 20

#Apply mean filter with each structuring element
smoothed_square = filters.rank.mean(coins_image, selem_square)
smoothed_rectangle = filters.rank.mean(coins_image, selem_rectangle)
smoothed_star = filters.rank.mean(coins_image, selem_star)
smoothed_diamond = filters.rank.mean(coins_image, selem_diamond)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

#Display structuring elements and corresponding smoothed images
axes[0, 0].imshow(selem_square, cmap='gray')
axes[0, 0].set_title("Square SELEM")
axes[0, 0].axis('off')

axes[0, 1].imshow(selem_rectangle, cmap='gray')
axes[0, 1].set_title("Rectangle SELEM")
axes[0, 1].axis('off')

axes[0, 2].imshow(selem_star, cmap='gray')
axes[0, 2].set_title("Star SELEM")
axes[0, 2].axis('off')

axes[0, 3].imshow(selem_diamond, cmap='gray')
axes[0, 3].set_title("Diamond SELEM")
axes[0, 3].axis('off')

# Row 2: Show the smoothed images
axes[1, 0].imshow(smoothed_square, cmap='gray')
axes[1, 0].set_title("Smoothed (Square SELEM)")
axes[1, 0].axis('off')

axes[1, 1].imshow(smoothed_rectangle, cmap='gray')
axes[1, 1].set_title("Smoothed (Rectangle SELEM)")
axes[1, 1].axis('off')

axes[1, 2].imshow(smoothed_star, cmap='gray')
axes[1, 2].set_title("Smoothed (Star SELEM)")
axes[1, 2].axis('off')

axes[1, 3].imshow(smoothed_diamond, cmap='gray')
axes[1, 3].set_title("Smoothed (Diamond SELEM)")
axes[1, 3].axis('off')

plt.tight_layout()
plt.show()

*   How does changing the radius of disk affect the smoothing functionality?

**Solution**

By changing the radius of the disk structuring element, or selem, it directly affects the extent of smoothing applied to the image. A larger radius includes more neighboring pixels in the mean calculation, resulting in stronger smoothing and a blurrier image. Conversely, a smaller radius focuses on fewer neighboring pixels, leading to a less pronounced smoothing effect, maintaining more detail. That is to say that increasing the radius smooths the image more, while decreasing it preserves finer details but reduces the level of smoothing.


*   What is the observed behaviour with difference in the structuring element?



**Solution**

The choice of structuring element (selem) significantly impacts the smoothing behavior.

- Square: Provides uniform smoothing in all directions, creating a balanced blur with no directional preference.
- Rectangle: Smooths more in the direction of the longer side, leading to a less uniform blur, often preserving detail along the shorter side.
- Star: Offers a more localized smoothing effect with a focus on pixels within the radius, preserving edges and details more than the square and rectangle in some areas.
- Diamond: Similar to the star, but with a more angular smoothing pattern, affecting pixel neighborhoods differently, leading to less uniform smoothing near the edges.
  
In general, the shape and orientation of the structuring element influence how the filter smooths the image, with rectangular and star shapes often introducing directional smoothing effects.



*   What is the difference between mean filter and gaussian filter?
*   Where do you use mean filters and where do you use gaussian filters?



**Solution**

A mean filter is a simple averaging filter that replaces each pixel's value with the average of its neighboring pixels, offering uniform smoothing. It's effective for removing salt-and-pepper noise but can blur edges and details. A Gaussian filter uses a weighted average where pixels closer to the center of the kernel have higher weights, following a Gaussian distribution. This filter provides smoother results while better preserving edges, making it ideal for reducing Gaussian noise and for applications where edge preservation is crucial, such as image preprocessing in object detection or segmentation. While mean filters are useful in scenarios with uniform noise, Gaussian filters are preferred when maintaining detail and smoothness without sacrificing edge clarity is important.